## Import libraries

In [1]:
from helper import get_paths, get_data, _embed

import numpy as np
import pandas as pd
import random
import os

import seaborn as sns
import matplotlib.pyplot as plt

from scipy import signal
from scipy.integrate import simps
from scipy.stats import entropy
from scipy.signal import periodogram, welch
from sklearn.metrics import auc, plot_roc_curve
from sklearn.model_selection import StratifiedKFold

from sklearn import metrics
from sklearn import svm

random_state = np.random.RandomState(0)

## Features
* Shannon Entropy
* Spectral Entropy
* Permutation Entropy
* Average power in Delta Band
* Average power in Theta Band

In [5]:
def shannon_entropy(x):
    counts = x.value_counts()
    she = entropy(counts, axis =0)
    return she

def spectral_entropy(x, sf, method='fft', nperseg=None, normalize=False):
    x = np.array(x)
    
    if method == 'fft':
        _, psd = periodogram(x, sf)
    elif method == 'welch':
        _, psd = welch(x, sf, nperseg=nperseg)
        
    psd_norm = np.divide(psd, psd.sum())
    se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
    
    return se

def perm_entropy(x, order=3, delay=1, normalize=False):
    x = np.array(x)
    ran_order = range(order)
    hashmult = np.power(order, ran_order)
    
    # Embed x and sort the order of permutations
    sorted_idx = _embed(x, order=order, delay=delay).argsort(kind='quicksort')
    
    # Associate unique integer to each permutations
    hashval = (np.multiply(sorted_idx, hashmult)).sum(1)
    
    # Return the counts
    _, c = np.unique(hashval, return_counts=True)
    
    # Use np.true_divide for Python 2 compatibility
    p = np.true_divide(c, c.sum())
    pe = -np.multiply(p, np.log2(p)).sum()

    return pe

def band_power(x, low, high, sf = 173.61):
    win = 4 * sf
    freqs, psd = signal.welch(x, sf, nperseg=win)

    # Find intersecting values in frequency vector
    idx_delta = np.logical_and(freqs >= low, freqs <= high)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]  

    # Compute the absolute power by approximating the area under the curve
    power_val = simps(psd[idx_delta], dx=freq_res)
    return power_val

In [6]:
def get_features(paths):
    features = []
    for file in paths:
        with open(file) as f:
            data_oi = pd.Series([int(val) for val in f.read().splitlines()])
            
            she = shannon_entropy(data_oi)
            se = spectral_entropy(data_oi, sf= 173.61)
            perme = perm_entropy(data_oi)

            delta_power_val = band_power(data_oi, low = 0.5, high = 4)
            theta_power_val = band_power(data_oi, low = 4.01, high = 8)

            features.append([she, se, perme, delta_power_val, theta_power_val])
            
    return np.array(features)

## Load Data
**SET A** (Z.zip)--> healthy, eyes open 

**SET B** (O.zip)--> healthy, eyes closed

**SET C** (N.zip)--> epileptic in the epileptogenic zone, during non-seizure time

**SET D** (F.zip)--> epileptic in hippocampus, during non-seizure time

**SET E** (S.zip)--> epileptic, only contained seizure activity

In [7]:
tempA = get_paths("data/Z/")
tempB = get_paths("data/O/")
tempC = get_paths("data/N/")
tempD = get_paths("data/F/")
tempE = get_paths("data/S/")

In [8]:
all_paths = tempA + tempB + tempC + tempD + tempE
X = get_features(all_paths)
y = np.array([0]*200+[1]*300)

## Plot Results

In [ ]:
cv = StratifiedKFold(n_splits=6)
classifier = svm.SVC(kernel='linear', probability=True,
                     random_state=random_state)

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

fig, ax = plt.subplots(figsize=(8,8))
for i, (train, test) in enumerate(cv.split(X, y)):
    classifier.fit(X[train], y[train])
    viz = plot_roc_curve(classifier, X[test], y[test],
                         name='ROC fold {}'.format(i),
                         alpha=0.3, lw=1, ax=ax)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='b',
        label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="Linear SVM ROC Curve")
ax.legend(loc="lower right", fontsize = 'small')
plt.show()